<a href="https://colab.research.google.com/github/UmardrazArshad/Quarter_02_PROJECTS/blob/main/3_LangChain_Tool_Calling_Calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **I Integrate this chain into a web application using Streamlit.**
**web app link** : https://umardrazarshad-1st-project-gemini-app-tlasvu.streamlit.app/

In [1]:
!pip install langchain google-generativeai langchain_google_genai -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 877.3 kB/s eta 0:00:00


In [2]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

In [3]:
from langchain_core.tools import tool
import requests
@tool
def calculator(expression: str) -> float:
    """
    Evaluate a mathematical expression and return the result.

    Parameters:
    expression (str): A string containing the mathematical expression to evaluate.

    Returns:
    float: The result of the evaluated expression.

    Examples:
    >>> evaluate_expression("2 + 3 * 4")
    14.0
    >>> evaluate_expression("(10 / 2) + 8")
    13.0

    Note:
    - This function uses Python's `eval()` to calculate the result.
    - Ensure the input is sanitized to avoid malicious code execution.
    """
    try:
        # Evaluate the expression safely
        result = eval(expression, {"__builtins__": {}})
        return float(result)
    except Exception as e:
        print(f"Error evaluating expression: {e}")
        return None


@tool
def get_stock_price(symbol: str) -> str:
    """Fetches the current stock price of a company based on its stock symbol using the Polygon API.

    Args:
        symbol (str): The stock symbol of the company (e.g., 'AAPL' for Apple, 'GOOGL' for Google).

    Returns:
        str: A message containing the current stock price of the company.

    Raises:
        HTTPError: If the HTTP request to the stock API fails (e.g., 404 or 500 status).
        RequestException: If there is an issue with the request itself (e.g., connection error).
        Exception: For any other unexpected errors during the execution of the function.

    """
    api_key = "2bx0DyQuypHfwohF46294_29KpFtMKzt"  # Replace this with your actual secret API key from Polygon
    url = f"https://api.polygon.io/v2/aggs/ticker/{symbol}/prev"  # Polygon endpoint for previous close price

    try:
        # Send a GET request with the API key
        response = requests.get(url, params={'apiKey': api_key})
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx, 5xx)

        # Assuming the data contains 'close' in the response for the last closing price
        data = response.json()
        price = data.get('results', [{}])[0].get('c')  # 'c' is the closing price

        if price:
            return f"Tool used: get_stock_price\n get_stock_price tool is used to find The current price of {symbol} is ${price}"
        else:
            return f"Error: Could not retrieve stock data for {symbol}.\nTool used: get_stock_price"

    except requests.exceptions.HTTPError as http_err:
        return f"HTTP error occurred: {http_err}\nTool used: get_stock_price"
    except requests.exceptions.RequestException as req_err:
        return f"Request error occurred: {req_err}\nTool used: get_stock_price"
    except Exception as err:
        return f"An unexpected error occurred: {err}\nTool used: get_stock_price"


In [4]:
tools = [calculator,get_stock_price]

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model = "gemini-2.0-flash-exp" , api_key=GOOGLE_API_KEY)

In [6]:
from langchain.agents import initialize_agent, AgentType

In [7]:
agent = initialize_agent(tools, llm , agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION )

<ipython-input-7-a14b04c8fa12>:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(tools, llm , agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION )


In [8]:
response =  agent.invoke({"input":"what is 2 multiply by 3"})
response["output"]

'2 multiplied by 3 is 6.'

In [9]:
response =  agent.invoke({"input":"what is stock price of tata"})
response["output"]

'The current stock price of Tata Motors (TTM) is $25.14.'